<a href="https://colab.research.google.com/github/VIOLDAVE/my_project/blob/main/Massive_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project requirement: https://docs.google.com/document/d/1GkK28wOyjTPZEZuOumKPU0z1NzVT_9sxETjPOcLPVYo/edit?tab=t.0#heading=h.qifoo7co6qtd

Professor : https://malchiodi.di.unimi.it/teaching/AMD-DSE/2024-25/en

In [ ]:
import os

if not os.path.exists("my_project"):
    !git clone https://github.com/VIOLDAVE/my_project.git

if os.path.exists("my_project"):
    os.chdir("my_project")
else:
    raise FileNotFoundError("Failed to find or clone the 'my_project' directory.")

Cloning into 'my_project'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 3), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), done.
Resolving deltas: 100% (3/3), done.


Install packages.

In [ ]:
!pip install -r requirement.txt

 kaggle  set up and Authentication

In [42]:
import json, os
from google.colab import files

# Upload the file
uploaded = files.upload()

# Process the uploaded file
for fn in uploaded.keys():
    print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

    # Read credentials from the uploaded file
    with open(fn, 'r') as f:
        creds = json.load(f)
        os.environ['KAGGLE_USERNAME'] = 'violaawor2'
        os.environ['KAGGLE_KEY'] = 'f93b76093cd1863d38f50e4da7642437'

    # Move and rename the uploaded file to the correct kaggle.json path
    os.makedirs("/root/.kaggle", exist_ok=True)
    os.rename(fn, "/root/.kaggle/kaggle.json")
    os.chmod("/root/.kaggle/kaggle.json", 0o600)


Saving kaggle_2.json to kaggle_2 (4).json
User uploaded file "kaggle_2 (4).json" with length 66 bytes


dowloading files

In [43]:
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 99% 1.05G/1.06G [00:12<00:00, 258MB/s]
100% 1.06G/1.06G [00:12<00:00, 94.6MB/s]


In [47]:
# Load the dataset with spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()
books_rating = spark.read.csv("/content/data/Books_rating.csv", header=True, inferSchema=True)

Unzipping file

In [46]:
import zipfile
import os

zip_file_path = "/content/amazon-books-reviews.zip"
extract_path = "/content/data"

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")

# List the extracted files to confirm
!ls /content/data

Files extracted to: /content/data
books_data.csv	Books_rating.csv


Loading the dataset

In [48]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()
books_rating = spark.read.csv("/content/data/Books_rating.csv", header=True, inferSchema=True)

Show a few raws of the dataset

In [49]:
books_rating.show(5)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| NULL| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/7|         4.0|  940636800|Nice collection o...|This is only for ...|
|0826414346|Dr. Seuss: Americ...| NULL|A30TK6U7DNS82R|       Kevin Killian|             10/10|         5.0| 1095724800|   Really Enjoyed It|I don't care much...|
|0826414346|Dr. Seuss: Americ...| NULL|A3UH4UZ4RSVO82|        John Granger|             10/11|         5.0| 1078790400|Essential for eve...|"If people become...|
|0826414346|Dr. Seuss: Ameri

creating a sample of dataset becase the file is too heavy for my colab space

In [51]:
sample = 0.02# can be adjusted as needed

In [55]:
books_sample = books_rating.sample(fraction=sample, seed=42)
books_sample.show(10)
size = books_sample.count()

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|0595344550|Whispers of the W...|10.95| AUR0VA5H0C66C|"LoveToRead ""Act...|               1/2|         1.0| 1119225600|        Buyer beware|"This is a self-p...|
|B000NKGYMK|    Alaska Sourdough| NULL|A258YNWJW2264M|"Tessa F. Briggs ...|              8/14|         3.0| 1241827200|Not your quick re...|"After having a c...|
|B000NKGYMK|    Alaska Sourdough| NULL|A2WY5VMJQ0MM1A|                 Aoy|               0/0|         5.0| 1351641600|        Old and good|This book is wort...|
|157067051X|Voices from the 

In [56]:
size

60498

Checking null valuse,. I will use the review column which has strings.

In [59]:
# Checking  for null values in the 'review/text' column
null_df = books_rating.filter(
    books_rating['review/text'].isNull() | (books_rating['review/text'].rlike('^\s*$'))
)

null_df.show(15)


+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|        Id|               Title|               Price|             User_id|         profileName|  review/helpfulness|        review/score|         review/time|      review/summary|review/text|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|B00085T7O2|"""Catch 'em aliv...|",,ASEH0CVYVGZCZ,...|                 0/1|                 5.0|          1341705600|Fascinating life ...|There are many id...|                NULL|       NULL|
|B00085T7O2|"""Catch 'em aliv...|",,A91QQ7IYBAK2Q,...| terrible forward...|I found the book ...|                NULL|                NULL|                NULL|                NULL|       NULL|
|B0007DMFZS|"""Beloved friend...|",

since there are essentially copies of the'review/text' null entries.

 Aside from the Title and the IDs, they have the same columns.  A qualitative examination reveals that they are about the same book even though the titles are different.

There are numerous other reviews of Lord of the Rings.  I believe that the NA values should be retained because they cannot be discarded.  The title can also be used as a similarity check.

In [60]:
from pyspark.sql.functions import col
filter_df = books_rating.filter(
    col("Title").contains("Lord of the Rings")
).select("Title", "review/text").limit(100)

# Show the result
filter_df.show(100, truncate=True)

+--------------------+--------------------+
|               Title|         review/text|
+--------------------+--------------------+
|Myth & Middle-Ear...|&quot;As a long-t...|
|Myth & Middle-Ear...|I just finished r...|
|Myth & Middle-Ear...|Juggles large num...|
|The Lord of the R...|"I enjoy these ta...|
|The Lord of the R...|This drama comes ...|
|The Return of the...|Have the Rings fo...|
|The Return of the...|I highly recommen...|
|Gollum: How We Ma...|"Andy Serkis is t...|
|Gollum: How We Ma...|I guess I'm a bit...|
|The Lord of the R...|I've never attemp...|
|The Lord of the R...|This is one of th...|
|The Lord of the R...|This made an idea...|
|The Lord of the R...|This is one hefty...|
|The Lord of the R...|This book is a mu...|
|The Lord of the R...|This comprehensiv...|
|The Lord of the R...|I just got the bo...|
|The Lord of the R...|"Length:: 2:05 Mi...|
|The Lord of the R...|".""...has left t...|
|The Lord of the R...|I will not belabo...|
|The Lord of the R...|"It is ama

In [63]:
# Check for null values in the 'review/text' column
Subsample_book = books_sample.filter(
    books_sample['review/text'].isNull() | (books_rating_sample['review/text'].rlike('^\s*$'))
)

Subsample_book.show(10)

+----------+--------------------+-----+-------+-----------+------------------+------------+-----------+--------------------+-----------+
|        Id|               Title|Price|User_id|profileName|review/helpfulness|review/score|review/time|      review/summary|review/text|
+----------+--------------------+-----+-------+-----------+------------------+------------+-----------+--------------------+-----------+
|B000I1VJLA|The Lord of the R...| NULL|   NULL|       NULL|               0/1|         5.0|  938563200|have only one wor...|       NULL|
+----------+--------------------+-----+-------+-----------+------------------+------------+-----------+--------------------+-----------+

